In [1]:
import sys
sys.path.append('/home/oleg/.python/lib/python3.11/site-packages/')
from datetime import datetime
import pickle
import numpy as np
import pandas as pd
import sklearn
from sklearn import cluster
# import tensorflow as tf
import transformers
import torch

In [2]:
print('Python', sys.version)
modules = [np, pd, sklearn, transformers, torch]
for module in modules:
    print(module.__name__, module.__version__)

Python 3.11.6 (main, Nov 14 2023, 09:36:21) [GCC 13.2.1 20230801]
numpy 1.26.1
pandas 1.5.3
sklearn 1.3.2
transformers 4.33.1
torch 2.1.0


In [3]:
df_data = pd.read_csv('train_dataset_train.csv', sep=';')
df_data

,Исполнитель,Группа тем,Текст инцидента,Тема
0,Лысьвенский городской округ,Благоустройство,"'Добрый день. Сегодня, 20.08.22, моя мать шла ...",★ Ямы во дворах
1,Министерство социального развития ПК,Социальное обслуживание и защита,"'Пермь г, +79194692145. В Перми с ноября 2021 ...",Оказание гос. соц. помощи
2,Министерство социального развития ПК,Социальное обслуживание и защита,'Добрый день ! Скажите пожалуйста если подовал...,Дети и многодетные семьи
3,Город Пермь,Общественный транспорт,'Каждая из них не о чем. Люди на остановках хо...,Содержание остановок
4,Министерство здравоохранения,Здравоохранение/Медицина,'В Березниках у сына привитого откоронавируса ...,Технические проблемы с записью на прием к врачу
...,...,...,...,...
23123,Губахинский городской округ,ЖКХ,'Город Гремячинск-— ситуация с теплом на улице...,Ненадлежащее качество или отсутствие отопления
23124,Министерство здравоохранения,Здравоохранение/Медицина,"'Здравствуйте у меня ребёнку 2 месяца , тест н...",Технические проблемы с записью на прием к врачу
23125,Лысьвенский городской округ,Благоустройство,'А что творится с благоустройством дворов! Воо...,Благоустройство придомовых территорий
23126,Лысьвенский городской округ,ЖКХ,'Сообщение без текста,Ненадлежащее качество или отсутствие отопления


In [5]:
from math import ceil
int(ceil(len(df_data)/10))

2313

In [4]:
s_themes = df_data['Тема'].value_counts()
s_themes

Дети и многодетные семьи                                                                                           2008
Оказание гос. соц. помощи                                                                                          1886
★ Нарушение правил очистки дорог от снега и наледи/Обращения о необходимости очистить тротуар от снега и наледи    1269
Технические проблемы с записью на прием к врачу                                                                     941
★ Оказание медицинской помощи не в полном объеме/отказ в оказании медицинской помощи                                885
                                                                                                                   ... 
Выбросы вредных веществ в атмосферу/загрязнение воздуха                                                               8
Обработка и уничтожение грызунов (дератизация)                                                                        8
Установка ограждений, препятствующих въе

In [5]:
s_resp = df_data['Исполнитель'].value_counts()
s_resp

Министерство здравоохранения                          5765
Город Пермь                                           4550
Министерство социального развития ПК                  4386
Лысьвенский городской округ                           2073
ИГЖН ПК                                               1861
АО ПРО ТКО                                            1514
Губахинский городской округ                           1097
Александровский муниципальный округ Пермского края     979
Министерство образования                               520
Бардымский муниципальный округ Пермского края          383
Name: Исполнитель, dtype: int64

In [6]:
s_group = df_data['Группа тем'].value_counts()
s_group

Здравоохранение/Медицина                    4673
Социальное обслуживание и защита            4626
Дороги                                      3132
ЖКХ                                         2703
Благоустройство                             2529
Мусор/Свалки/ТКО                            1735
Общественный транспорт                       869
Коронавирус                                  810
Образование                                  701
Безопасность                                 299
Связь и телевидение                          236
Мобилизация                                  134
Физическая культура и спорт                  120
Строительство и архитектура                  113
Спецпроекты                                   84
Газ и топливо                                 83
Культура                                      69
Электроснабжение                              37
Экономика и бизнес                            30
Экология                                      25
Памятники и объекты 

In [7]:
class ProgressMonitor():
    # def start(start=0, stop, step=10):
    def __init__(self, *args):
        import pandas as pd
        start, stop, step = 0, 100, 10
        if len(args) == 3:
            start, stop, step = args
        elif len(args) == 2:
            start, stop = args
        elif len(args) == 1:
            stop = args[0]
            
        self.start = start
        self.stop = stop
        self.step = step
        self.qty = stop - start
        self.start_time = pd.Timestamp.now()
        self.last_percent = 0
        self.last_time = self.start_time
        print('Начало', self.start_time.strftime('%d.%m.%Y %X'))

    def update(self, param):
        import pandas as pd
        percent = 100*(param - self.start)//self.qty
        if percent >= self.last_percent + self.step:
            cur_time = pd.Timestamp.now()
            print(f'{percent: 3d}% {cur_time.strftime("%d.%m.%Y %X")} {cur_time - self.last_time}')
            self.last_percent = percent
            self.last_time = cur_time
            
    def finish(self):
        import pandas as pd
        cur_time = pd.Timestamp.now()
        print('Завершено', cur_time.strftime('%d.%m.%Y %X'))
        print('Продолжительность', cur_time - self.start_time)

In [8]:
from transformers import AutoTokenizer, AutoModel
sbert_t = AutoTokenizer.from_pretrained("ai-forever/sbert_large_mt_nlu_ru")
sbert_m = AutoModel.from_pretrained("ai-forever/sbert_large_mt_nlu_ru")

2023-11-23 20:42:02.773577: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
prog_mon = ProgressMonitor(len(df_data))
outputs = []
for i, release in enumerate(df_data['Текст инцидента']):
    encoded_input = sbert_t(release, padding=True, truncation=True, max_length=512, return_tensors='pt')
    with torch.no_grad():
        model_output = sbert_m(**encoded_input)
    outputs.append(model_output[0][0].numpy())
    prog_mon.update(i)
# outputs = np.array(outputs)
prog_mon.finish()

Начало 23.11.2023 20:43:51
 10% 23.11.2023 21:04:00 0 days 00:20:08.751947
 20% 23.11.2023 21:26:50 0 days 00:22:49.539602
 30% 23.11.2023 21:49:10 0 days 00:22:20.649039
 40% 23.11.2023 22:11:53 0 days 00:22:42.770210
 50% 23.11.2023 22:36:05 0 days 00:24:11.725107
 60% 23.11.2023 23:00:00 0 days 00:23:54.746115
 70% 23.11.2023 23:20:05 0 days 00:20:05.427534
 80% 23.11.2023 23:41:25 0 days 00:21:20.139058
 90% 24.11.2023 00:02:01 0 days 00:20:36.215255
Завершено 24.11.2023 00:23:36
Продолжительность 0 days 03:39:44.251733


In [10]:
outputs[0].shape

(103, 1024)

In [11]:
outputs[1].shape

(61, 1024)

In [15]:
lens = list(map(len, outputs))
lens

In [16]:
max(lens), min(lens)

ValueError: min() arg is an empty sequence

___

In [1]:
import sys
sys.path.append('/home/oleg/.python/lib/python3.11/site-packages/')
from datetime import datetime
import pickle
import numpy as np
import pandas as pd
import sklearn
from sklearn import cluster
# import tensorflow as tf
# import transformers
# import torch

In [2]:
with open('sbert_0.pickle', 'rb') as f:
    output0 = pickle.load(f)

In [3]:
with open('sbert_1.pickle', 'rb') as f:
    output1 = pickle.load(f)

In [4]:
outputs = output0 + output1

In [5]:
del output0, output1

In [6]:
len(outputs)

9252

In [7]:
lens = list(map(len, outputs))

In [8]:
max(lens), min(lens)

(512, 3)

In [9]:
sum(lens)

542541

In [10]:
df_data = pd.read_csv('train_dataset_train.csv', sep=';')

In [11]:
s_themes = df_data['Тема'].value_counts()
s_themes

Дети и многодетные семьи                                                                                           2008
Оказание гос. соц. помощи                                                                                          1886
★ Нарушение правил очистки дорог от снега и наледи/Обращения о необходимости очистить тротуар от снега и наледи    1269
Технические проблемы с записью на прием к врачу                                                                     941
★ Оказание медицинской помощи не в полном объеме/отказ в оказании медицинской помощи                                885
                                                                                                                   ... 
Выбросы вредных веществ в атмосферу/загрязнение воздуха                                                               8
Обработка и уничтожение грызунов (дератизация)                                                                        8
Установка ограждений, препятствующих въе

In [12]:
s_group = df_data['Группа тем'].value_counts()
s_group

Здравоохранение/Медицина                    4673
Социальное обслуживание и защита            4626
Дороги                                      3132
ЖКХ                                         2703
Благоустройство                             2529
Мусор/Свалки/ТКО                            1735
Общественный транспорт                       869
Коронавирус                                  810
Образование                                  701
Безопасность                                 299
Связь и телевидение                          236
Мобилизация                                  134
Физическая культура и спорт                  120
Строительство и архитектура                  113
Спецпроекты                                   84
Газ и топливо                                 83
Культура                                      69
Электроснабжение                              37
Экономика и бизнес                            30
Экология                                      25
Памятники и объекты 

In [13]:
s_contractor = df_data['Исполнитель'].value_counts()
s_contractor

Министерство здравоохранения                          5765
Город Пермь                                           4550
Министерство социального развития ПК                  4386
Лысьвенский городской округ                           2073
ИГЖН ПК                                               1861
АО ПРО ТКО                                            1514
Губахинский городской округ                           1097
Александровский муниципальный округ Пермского края     979
Министерство образования                               520
Бардымский муниципальный округ Пермского края          383
Name: Исполнитель, dtype: int64

In [14]:
len(s_themes), len(s_group), len(s_contractor)

(195, 26, 10)

In [15]:
# здесь надо посмотреть, однозначно ли тема входит в группу
grouped = df_data.groupby(['Тема', 'Группа тем'])['Группа тем'].count()
grouped

Тема                                         Группа тем                      
Аварийное жилье/переселение                  Социальное обслуживание и защита    234
Архитектура города                           Строительство и архитектура          20
Беженцы                                      Безопасность                         19
Безопасность образовательного процесса       Образование                          45
Безопасность общественных пространств        Безопасность                        101
                                                                                ... 
★ Протечки с кровли (системы водостока)      ЖКХ                                  81
★ Скорая помощь                              Здравоохранение/Медицина             87
★ Стихийные свалки в городе/в парках/в лесу  Мусор/Свалки/ТКО                     52
★ Уборка/Вывоз мусора                        Мусор/Свалки/ТКО                    482
★ Ямы во дворах                              Благоустройство            

In [16]:
#  и в исполнителя
grouped = df_data.groupby(['Тема', 'Исполнитель'])['Исполнитель'].count()
grouped

Тема                         Исполнитель                                       
Аварийное жилье/переселение  Александровский муниципальный округ Пермского края     41
                             Город Пермь                                           107
                             Губахинский городской округ                            36
                             ИГЖН ПК                                                 7
                             Лысьвенский городской округ                            43
                                                                                  ... 
★ Ямы во дворах              Бардымский муниципальный округ Пермского края           1
                             Город Пермь                                             5
                             Губахинский городской округ                             1
                             ИГЖН ПК                                                 4
                             Лысьвенский городской

Количество строк в первом случае совпадает с количеством тем, значит, ни одна тема не повторяется, значит, входит только в одну группу.
Чего не скажешь об исполнителях.

1. Делаем список всех длин эмбеддинга для оставшихся данных
2. Получаем список индексов сэмплов, у которых длина эмбеддинга от 40 до 400
3. Из списка выбираем сэмплы а) чтобы были представлены все темы б) чтобы суммарная длина эмбеддингов была около 25к
4. Берёзим 25к векторов
5. Остальные вектора разбрасываем по кластерам предиктом
6. Считаем статистику по кластерам у каждого сэмпла, нормируем по общему количеству векторов на сэмпл
7. обучаем кэтбуст

In [17]:
lens = []
for i in range(5):
    with open(f'sbert_{i}.pickle', 'rb') as f:
        output0 = pickle.load(f)
    len0 = list(map(len, output0))
    lens = lens + len0
len(lens)

23128

In [19]:
df_data = df_data.iloc[:len(lens)]
df_data['lens'] = lens

/tmp/ipykernel_28991/2398763067.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['lens'] = lens


In [20]:
df_data

,Исполнитель,Группа тем,Текст инцидента,Тема,lens
0,Лысьвенский городской округ,Благоустройство,"'Добрый день. Сегодня, 20.08.22, моя мать шла ...",★ Ямы во дворах,103
1,Министерство социального развития ПК,Социальное обслуживание и защита,"'Пермь г, +79194692145. В Перми с ноября 2021 ...",Оказание гос. соц. помощи,61
2,Министерство социального развития ПК,Социальное обслуживание и защита,'Добрый день ! Скажите пожалуйста если подовал...,Дети и многодетные семьи,34
3,Город Пермь,Общественный транспорт,'Каждая из них не о чем. Люди на остановках хо...,Содержание остановок,86
4,Министерство здравоохранения,Здравоохранение/Медицина,'В Березниках у сына привитого откоронавируса ...,Технические проблемы с записью на прием к врачу,65
...,...,...,...,...,...
23123,Губахинский городской округ,ЖКХ,'Город Гремячинск-— ситуация с теплом на улице...,Ненадлежащее качество или отсутствие отопления,30
23124,Министерство здравоохранения,Здравоохранение/Медицина,"'Здравствуйте у меня ребёнку 2 месяца , тест н...",Технические проблемы с записью на прием к врачу,36
23125,Лысьвенский городской округ,Благоустройство,'А что творится с благоустройством дворов! Воо...,Благоустройство придомовых территорий,77
23126,Лысьвенский городской округ,ЖКХ,'Сообщение без текста,Ненадлежащее качество или отсутствие отопления,6


In [21]:
min_len = 17
max_len = 400
len_sum = df_data[(df_data.lens >= min_len)&(df_data.lens <= max_len)].lens.cumsum()
df_data['len_sum'] = len_sum
df_data

/tmp/ipykernel_28991/372938268.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['len_sum'] = len_sum


,Исполнитель,Группа тем,Текст инцидента,Тема,lens,len_sum
0,Лысьвенский городской округ,Благоустройство,"'Добрый день. Сегодня, 20.08.22, моя мать шла ...",★ Ямы во дворах,103,103.0
1,Министерство социального развития ПК,Социальное обслуживание и защита,"'Пермь г, +79194692145. В Перми с ноября 2021 ...",Оказание гос. соц. помощи,61,164.0
2,Министерство социального развития ПК,Социальное обслуживание и защита,'Добрый день ! Скажите пожалуйста если подовал...,Дети и многодетные семьи,34,198.0
3,Город Пермь,Общественный транспорт,'Каждая из них не о чем. Люди на остановках хо...,Содержание остановок,86,284.0
4,Министерство здравоохранения,Здравоохранение/Медицина,'В Березниках у сына привитого откоронавируса ...,Технические проблемы с записью на прием к врачу,65,349.0
...,...,...,...,...,...,...
23123,Губахинский городской округ,ЖКХ,'Город Гремячинск-— ситуация с теплом на улице...,Ненадлежащее качество или отсутствие отопления,30,1271328.0
23124,Министерство здравоохранения,Здравоохранение/Медицина,"'Здравствуйте у меня ребёнку 2 месяца , тест н...",Технические проблемы с записью на прием к врачу,36,1271364.0
23125,Лысьвенский городской округ,Благоустройство,'А что творится с благоустройством дворов! Воо...,Благоустройство придомовых территорий,77,1271441.0
23126,Лысьвенский городской округ,ЖКХ,'Сообщение без текста,Ненадлежащее качество или отсутствие отопления,6,NaN


In [22]:
df_data[df_data.len_sum.notna()]['Тема'].value_counts()

Дети и многодетные семьи                                                                                           1817
Оказание гос. соц. помощи                                                                                          1658
★ Нарушение правил очистки дорог от снега и наледи/Обращения о необходимости очистить тротуар от снега и наледи    1110
Технические проблемы с записью на прием к врачу                                                                     856
★ Оказание медицинской помощи не в полном объеме/отказ в оказании медицинской помощи                                830
                                                                                                                   ... 
Обработка и уничтожение грызунов (дератизация)                                                                        6
★ Повреждение опор ЛЭП                                                                                                6
Нарушение технологии и (или) сроков прои

In [23]:
# две темы выпали, надо вернуть - ничего не выпало
# хотя сначала надо сделать словарь тема - номер и проставить номера тем в датасете
s_themes

Дети и многодетные семьи                                                                                           2008
Оказание гос. соц. помощи                                                                                          1886
★ Нарушение правил очистки дорог от снега и наледи/Обращения о необходимости очистить тротуар от снега и наледи    1269
Технические проблемы с записью на прием к врачу                                                                     941
★ Оказание медицинской помощи не в полном объеме/отказ в оказании медицинской помощи                                885
                                                                                                                   ... 
Выбросы вредных веществ в атмосферу/загрязнение воздуха                                                               8
Обработка и уничтожение грызунов (дератизация)                                                                        8
Установка ограждений, препятствующих въе

In [24]:
theme_dict = pd.Series(df_data['Тема'].unique())
theme_dict

0                                        ★ Ямы во дворах
1                              Оказание гос. соц. помощи
2                               Дети и многодетные семьи
3                                   Содержание остановок
4        Технические проблемы с записью на прием к врачу
                             ...                        
190    Неисправность выступающих конструкций: балконо...
191                              Соц.обслуживание прочее
192                                     Устройство в ДОУ
193                                ★ Парковка на газонах
194                           Матери-одиночки, подростки
Length: 195, dtype: object

In [25]:
def themetovalue(theme):
    return theme_dict[theme_dict==theme].index[0]

df_data['theme'] = df_data['Тема'].apply(themetovalue)

/tmp/ipykernel_28991/775313707.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['theme'] = df_data['Тема'].apply(themetovalue)


In [26]:
group_dict = pd.Series(df_data['Группа тем'].unique())
def grouptovalue(group):
    return group_dict[group_dict==group].index[0]

contractor_dict = pd.Series(df_data['Исполнитель'].unique())
def contractortovalue(contractor):
    return contractor_dict[contractor_dict==contractor].index[0]

df_data['group'] = df_data['Группа тем'].apply(grouptovalue)
df_data['contractor'] = df_data['Исполнитель'].apply(contractortovalue)

/tmp/ipykernel_28991/4090774118.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['group'] = df_data['Группа тем'].apply(grouptovalue)
/tmp/ipykernel_28991/4090774118.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['contractor'] = df_data['Исполнитель'].apply(contractortovalue)


In [27]:
df_data

,Исполнитель,Группа тем,Текст инцидента,Тема,lens,len_sum,theme,group,contractor
0,Лысьвенский городской округ,Благоустройство,"'Добрый день. Сегодня, 20.08.22, моя мать шла ...",★ Ямы во дворах,103,103.0,0,0,0
1,Министерство социального развития ПК,Социальное обслуживание и защита,"'Пермь г, +79194692145. В Перми с ноября 2021 ...",Оказание гос. соц. помощи,61,164.0,1,1,1
2,Министерство социального развития ПК,Социальное обслуживание и защита,'Добрый день ! Скажите пожалуйста если подовал...,Дети и многодетные семьи,34,198.0,2,1,1
3,Город Пермь,Общественный транспорт,'Каждая из них не о чем. Люди на остановках хо...,Содержание остановок,86,284.0,3,2,2
4,Министерство здравоохранения,Здравоохранение/Медицина,'В Березниках у сына привитого откоронавируса ...,Технические проблемы с записью на прием к врачу,65,349.0,4,3,3
...,...,...,...,...,...,...,...,...,...
23123,Губахинский городской округ,ЖКХ,'Город Гремячинск-— ситуация с теплом на улице...,Ненадлежащее качество или отсутствие отопления,30,1271328.0,44,7,9
23124,Министерство здравоохранения,Здравоохранение/Медицина,"'Здравствуйте у меня ребёнку 2 месяца , тест н...",Технические проблемы с записью на прием к врачу,36,1271364.0,4,3,3
23125,Лысьвенский городской округ,Благоустройство,'А что творится с благоустройством дворов! Воо...,Благоустройство придомовых территорий,77,1271441.0,122,0,0
23126,Лысьвенский городской округ,ЖКХ,'Сообщение без текста,Ненадлежащее качество или отсутствие отопления,6,NaN,44,7,0


In [28]:
sapeth = 7    # samples per theme
class_qty = len(theme_dict)
samples_qty = len(df_data)
quantities = np.zeros((class_qty, ))
indices = np.zeros((class_qty*sapeth, ), dtype='int32')
cur_index = 0
for i, row in enumerate(df_data[df_data.len_sum.notna()].iloc):
    if quantities[row.theme] < sapeth:
        quantities[row.theme] += 1
        indices[cur_index] = row.name
        cur_index +=1
        if row.name > samples_qty:
            print(i, row)
            break
    if cur_index >= class_qty*sapeth:
        break
res = df_data.theme.loc[indices].value_counts().sort_index()
res

0      17
1       7
2       7
3       7
4       7
       ..
190     7
191     6
192     7
193     7
194     7
Name: theme, Length: 195, dtype: int64

In [29]:
df_data.loc[indices].lens.sum()

87012

In [30]:
df_data.fillna(0, inplace=True)

with open('indices.pickle', 'wb') as f:
    pickle.dump(indices, f)

df_data.to_csv('data.csv')

/tmp/ipykernel_28991/1693636473.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data.fillna(0, inplace=True)


___

In [1]:
import numpy as np
import pandas as pd
from sklearn import cluster
import pickle

In [2]:
# df_data = pd.read_csv('data.csv')
with open('indices.pickle', 'rb') as f:
    indices = pickle.load(f)

In [3]:
outputs = []
for i in range(5):
    with open(f'sbert_{i}.pickle', 'rb') as f:
        outputs += pickle.load(f)

In [4]:
new_outputs = []
for index in indices:
    new_outputs.append(outputs[index])

new_outputs = np.concatenate(new_outputs)
new_outputs.shape

(87012, 1024)

In [5]:
del outputs

In [6]:
start_time = pd.Timestamp.now()
print(start_time.strftime('%d.%m.%Y %X'))
print()
n_clusters = 195*3
kmeans = cluster.MiniBatchKMeans(n_clusters=n_clusters, random_state=0,
                                 n_init="auto", batch_size=n_clusters*10,
                                 max_iter=5000,
                                )
kmeans.fit(new_outputs)
stop_time = pd.Timestamp.now()
print('Готово', stop_time.strftime('%d.%m.%Y %X'))
print(stop_time-start_time)
print(kmeans.inertia_)

24.11.2023 15:34:47

Готово 24.11.2023 15:36:59
0 days 00:02:12.964754
7909832.0


In [7]:
kmeans.n_iter_

4

In [8]:
with open('kmeans.pickle', 'wb') as f:
    pickle.dump(kmeans, f)

___

1. загружаем
2. делаем массив ответов
3. по сэмплам - считаем кластеры, делим на длину эмбеддинга
4. тренируем кэтбуст

In [1]:
import numpy as np
import pandas as pd
from sklearn import cluster
import pickle

In [2]:
df_data = pd.read_csv('data.csv')
ys = df_data.theme
del df_data
len(ys)

23128

In [3]:
outputs = []
for i in range(5):
    with open(f'sbert_{i}.pickle', 'rb') as f:
        outputs += pickle.load(f)
len(outputs)

23128

In [4]:
class ProgressMonitor():
    # def start(start=0, stop, step=10):
    def __init__(self, *args):
        import pandas as pd
        start, stop, step = 0, 100, 10
        if len(args) == 3:
            start, stop, step = args
        elif len(args) == 2:
            start, stop = args
        elif len(args) == 1:
            stop = args[0]
            
        self.start = start
        self.stop = stop
        self.step = step
        self.qty = stop - start
        self.start_time = pd.Timestamp.now()
        self.last_percent = 0
        self.last_time = self.start_time
        print('Начало', self.start_time.strftime('%d.%m.%Y %X'))

    def update(self, param):
        import pandas as pd
        percent = 100*(param - self.start)//self.qty
        if percent >= self.last_percent + self.step:
            cur_time = pd.Timestamp.now()
            print(f'{percent: 3d}% {cur_time.strftime("%d.%m.%Y %X")} {cur_time - self.last_time}')
            self.last_percent = percent
            self.last_time = cur_time
            
    def finish(self):
        import pandas as pd
        cur_time = pd.Timestamp.now()
        print('Завершено', cur_time.strftime('%d.%m.%Y %X'))
        print('Продолжительность', cur_time - self.start_time)

In [5]:
with open('kmeans.pickle', 'rb') as f:
    kmeans = pickle.load(f)

In [6]:
# готовим датасет для классификатора
# X у нас будет номера кластеров для всех векторов из эмбеддинга - нет
# уменьшим размерность сэмпла - вместо всего списка
#    номеров кластеров дадим количество векторов в конкретном кластере
# 1. предиктуем сэмпл - получаем список кластеров
# 2. считаем количество кластеров, делим на длину сэмпла
# придётся каждый массив с кластерами преобразовывать в серию, поскольку длины у них разные
sample_qty = len(ys)
cluster_qty = 159
prog_mon = ProgressMonitor(sample_qty)
xs = pd.DataFrame(np.zeros((cluster_qty, sample_qty, )))
for i in range(sample_qty):
    clusters = kmeans.predict(outputs[i])
    xs[i] = pd.Series(clusters).value_counts(normalize=True)
    prog_mon.update(i)
xs = xs.fillna(0).T
prog_mon.finish()

Начало 24.11.2023 16:10:09
 10% 24.11.2023 16:11:29 0 days 00:01:20.327880
 20% 24.11.2023 16:12:50 0 days 00:01:21.543378
 30% 24.11.2023 16:14:10 0 days 00:01:19.076991
 40% 24.11.2023 16:15:27 0 days 00:01:17.941935
 50% 24.11.2023 16:16:42 0 days 00:01:14.687515
 60% 24.11.2023 16:17:56 0 days 00:01:14.092916
 70% 24.11.2023 16:19:10 0 days 00:01:14.168944
 80% 24.11.2023 16:20:24 0 days 00:01:13.667676
 90% 24.11.2023 16:21:34 0 days 00:01:09.965870
Завершено 24.11.2023 16:22:45
Продолжительность 0 days 00:12:36.524376


In [7]:
del outputs

In [8]:
train_qty = int(0.8 * sample_qty)
train_xs = xs.iloc[:train_qty]
test_xs = xs.iloc[train_qty:]
train_ys = ys.iloc[:train_qty]
test_ys = ys.iloc[train_qty:]
c1 = train_xs[train_ys==0].sum()/len(train_xs[train_ys==0])
c2 = test_xs[test_ys==0].sum()/len(test_xs[test_ys==0])
df_cmp = pd.DataFrame({'c1': c1, 'c2': c2})

In [9]:
df_cmp[df_cmp.c2 != 0]

,c1,c2
16,0.003213,0.005650
58,0.001058,0.189944
68,0.015410,0.011628
71,0.000000,0.003724
78,0.000000,0.039548
146,0.028571,0.034884


In [10]:
df_cmp[df_cmp.c1 != 0]

,c1,c2
8,0.038026,0.000000
16,0.003213,0.005650
31,0.001361,0.000000
34,0.008032,0.000000
50,0.119679,0.000000
58,0.001058,0.189944
68,0.015410,0.011628
79,0.066667,0.000000
104,0.005442,0.000000
124,0.003333,0.000000


In [11]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score, f1_score, make_scorer
from catboost import CatBoostClassifier
from catboost.metrics import MultiClassOneVsAll, F1

random_state = 64
X_train, X_test, y_train, y_test = train_test_split(xs.to_numpy(), ys.to_numpy(), test_size=0.1,
                                                    stratify=ys.to_numpy(), random_state=random_state)
cc = CatBoostClassifier(verbose=0, iterations=1000,    #, loss_function=MultiClassOneVsAll
                        random_state=random_state, use_best_model=True, depth=8,)    #  eval_metric=
print('Начали', pd.Timestamp.now().strftime('%X'))
cc.fit(X_train, y_train, eval_set=(X_test, y_test))
y_pred = cc.predict(X_test)
score = f1_score(y_test, y_pred, average='micro')
print(score)

print('Готово,', pd.Timestamp.now().strftime("%X"))

Начали 16:22:53
0.19152615650670127
Готово, 18:38:10


Вот это облом.
Можно, конечно, попробовать другие модели, но даже если и будет лучше, то не радикально.

In [13]:
with open('catboost.pickle', 'wb') as f:
    pickle.dump(cc, f)

In [22]:
df_data = pd.read_csv('data.csv')
df_group_theme = df_data.groupby(['theme', 'group']).group.count().rename('qty').reset_index()
df_group_theme

,theme,group,qty
0,0,0,18
1,1,1,1886
2,2,1,2008
3,3,2,132
4,4,3,941
...,...,...,...
190,190,7,12
191,191,1,8
192,192,5,15
193,193,0,9


In [23]:
def theme_to_group(theme):
    return df_group_theme.group.loc[theme]

In [34]:
y_true = pd.Series(y_test).apply(theme_to_group).to_numpy()
y_true.shape, y_test.shape

((2313,), (2313,))

In [35]:
pd.Series(y_pred.flat).apply(theme_to_group).to_numpy().shape

(2313,)

In [37]:
f1_score(y_true, pd.Series(y_pred.flat).apply(theme_to_group).to_numpy(), average='macro')

0.13612555442321606

In [38]:
f1_score(y_true, pd.Series(y_pred.flat).apply(theme_to_group).to_numpy(), average='micro')

0.4340683095546909

In [62]:
f1_score(y_true, pd.Series(y_pred.flat).apply(theme_to_group).to_numpy(), average='macro'),
f1_score(y_true, pd.Series(y_pred.flat).apply(theme_to_group).to_numpy(), average='micro')

ValueError: Found input variables with inconsistent numbers of samples: [2313, 4626]

In [40]:
f1_score(y_true, pd.Series(y_pred.flat).apply(theme_to_group).to_numpy(), average='weighted')

0.4059885024725448

In [41]:
f1_score(y_test, y_pred, average='weighted')

0.1560710633034064

In [49]:
sapecon = 20    # samples per contractor
class_qty = len(df_data.contractor.unique())
samples_qty = len(df_data)

quantities = np.zeros((class_qty, ))
indices = np.zeros((class_qty*sapecon, ), dtype='int32')
cur_index = 0
for i, row in enumerate(df_data[df_data.len_sum.notna()].iloc):
    if quantities[row.contractor] < sapecon:
        quantities[row.contractor] += 1
        indices[cur_index] = row.name
        cur_index +=1
        if row.name > samples_qty:
            print(i, row)
            break
    if cur_index >= class_qty*sapecon:
        break
    
outputs = []
for i in range(5):
    with open(f'sbert_{i}.pickle', 'rb') as f:
        outputs += pickle.load(f)
        
new_outputs = []
for index in indices:
    new_outputs.append(outputs[index])

new_outputs = np.concatenate(new_outputs)
print(new_outputs.shape)

(10849, 1024)


In [50]:
start_time = pd.Timestamp.now()
print(start_time.strftime('%d.%m.%Y %X'))

n_clusters = class_qty*3
kmean_c = cluster.MiniBatchKMeans(n_clusters=n_clusters, random_state=0,
                                  n_init="auto", batch_size=n_clusters*10,
                                  max_iter=5000,
                                 )
kmean_c.fit(new_outputs)
stop_time = pd.Timestamp.now()
print('Готово', stop_time.strftime('%d.%m.%Y %X'))
print(stop_time-start_time)
print(kmean_c.inertia_)

24.11.2023 22:38:43
Готово 24.11.2023 22:38:44
0 days 00:00:00.934496
1803585.375


In [51]:
sample_qty = len(df_data)

prog_mon = ProgressMonitor(sample_qty)
xs = pd.DataFrame(np.zeros((n_clusters, sample_qty, )))
for i in range(sample_qty):
    clusters = kmeans.predict(outputs[i])
    xs[i] = pd.Series(clusters).value_counts(normalize=True)
    prog_mon.update(i)
xs = xs.fillna(0).T
prog_mon.finish()

Начало 24.11.2023 22:55:05
 10% 24.11.2023 22:56:16 0 days 00:01:11.171539
 20% 24.11.2023 22:57:28 0 days 00:01:12.104631
 30% 24.11.2023 22:58:38 0 days 00:01:10.480227
 40% 24.11.2023 22:59:49 0 days 00:01:10.355693
 50% 24.11.2023 23:00:58 0 days 00:01:09.346172
 60% 24.11.2023 23:02:09 0 days 00:01:10.695017
 70% 24.11.2023 23:03:21 0 days 00:01:12.445938
 80% 24.11.2023 23:04:34 0 days 00:01:12.558557
 90% 24.11.2023 23:05:43 0 days 00:01:09.009809
Завершено 24.11.2023 23:06:54
Продолжительность 0 days 00:11:49.667258


In [52]:
del outputs

In [53]:
xs.shape

(23128, 30)

In [61]:
ys = df_data.contractor
X_train, X_test, y_train, y_test = train_test_split(xs.to_numpy(), ys.to_numpy(), test_size=0.2,
                                                    stratify=ys.to_numpy(), random_state=random_state)
cc_c = CatBoostClassifier(verbose=0, iterations=5000,    #, loss_function=MultiClassOneVsAll
                        random_state=random_state, use_best_model=True, depth=8,)    #  eval_metric=
print('Начали', pd.Timestamp.now().strftime('%X'))
cc_c.fit(X_train, y_train, eval_set=(X_test, y_test))
y_pred = cc_c.predict(X_test)
score = f1_score(y_test, y_pred, average='weighted')
print(score)

print('Готово,', pd.Timestamp.now().strftime("%X"))

Начали 00:18:47
0.18734939608527415
Готово, 00:24:12


In [60]:
score = f1_score(y_test, y_pred, average='weighted')
score

0.18670202029843222

In [58]:
with open('kmean_c.pickle', 'wb')as f:
    pickle.dump(kmean_c, f)
with open('cc_c.pickle', 'wb')as f:
    pickle.dump(cc_c, f)

Когда надо обработать массив информации
1. Готовим таблички: темы с номерами, группы с номерами, исполнители с номерами
2. Готовим табличку темы - группы
1. С помощью берта получаем эмбеддинги
2. Загружаем kmean, обрабатываем эмбеддинги, получаем кластеры
3. Считаем кластеры
4. Загружаем кэтбуст, по количеству кластеров делаем классификацию
5. Переводим номера тем в темы и группы, номера исполнителей - в исполнителей
kmean и кэтбуст для тем и исполнителей разные